**This notebook produces the synthetic rheology comparison figure.**

It relies on results from `Englacial Velocity 2D.ipynb`. This notebook just does the plotting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import sympy
from sympy import *
import pickle

In [ ]:
files = [
    'outputs_stored/20240627_172511_n4.0.pickle',
    'outputs_stored/20240627_172129_n3.0.pickle',
    'outputs_stored/20240627_171904_n2.0.pickle'
]

results = []
for file in files:
    with open(file, 'rb') as f:
        results.append(pickle.load(f))

### Velocity vs depth profile at end

In [ ]:
# Plot each layer solution at x=100e3 as a function of depth
fig, ax = plt.subplots(figsize=(4, 8))

plot_pos_x = 90e3

result_colors = ['tab:blue', 'tab:orange', 'tab:purple', 'tab:olive']
result_markers = ['X', 's', 'P', 'o']

for result_idx, res in enumerate(results):
    u_at_plot_pos = sympy.lambdify(res['z'], res['u'].subs(res['x'], plot_pos_x), modules='numpy')(res['zs'])

    for layer_idx in res['layer_solutions'].keys():
        if (layer_idx == 1):
            lbl = f'ODE Solutions for n = {res["n"]}'
        else:
            lbl = None
        ax.scatter([res['layer_solutions'][layer_idx].sol(plot_pos_x)], [res['layers_t0'][layer_idx](plot_pos_x)], label=lbl, c=result_colors[result_idx], marker=result_markers[result_idx])

    ax.plot(u_at_plot_pos*scipy.constants.year, res['zs'], linestyle='--', linewidth=1, c=result_colors[result_idx], label=f'True n = {res["n"]}')

ax.set_xlabel('Horizontal velocity [m/yr]')
ax.set_ylabel('z [m]')
ax.legend()
ax.grid()
plt.show()

### Stress vs strain

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

for result_idx, res in enumerate(results):
    ax.scatter(np.log10(res['eff_stress']), np.log10(res['du_dz_central_diff']), label=f'n = {res["n"]}', s=2, c=result_colors[result_idx])

ax.set_aspect(0.5)
ax.set_xlabel('log(effective stress)')
ax.set_ylabel('log(strain rate)')
ax.legend()
# Set the grid spacing to 1 on both axes
ax.set_xticks(np.arange(2, 5, 1))
ax.set_yticks(np.arange(-10, -2, 1))
ax.set_ylim(-10, -2)
ax.set_xlim(2, 5)
ax.grid()

### Combined figure

In [ ]:
fig = plt.figure(constrained_layout=True, facecolor='white', figsize=(12,5))
gs = fig.add_gridspec(len(results), 3, width_ratios=[0.4, 0.3, 0.3], wspace=0.1)

result_colors = ['tab:blue', 'tab:orange', 'tab:purple', 'tab:olive']
result_markers = ['X', 's', 'P', 'o']

# Horizontal velocity profiles
vel_plot_labels = ['(a)', '(b)', '(c)']
vel_profile_axes = []
norm = matplotlib.colors.LogNorm(vmin=0.5, vmax=170)
last_result_ax = None # Track the bottom axis for adding the layer legend
for idx, r in enumerate(results):
    X, Z = np.meshgrid(r['xs'], r['zs'])
    U = sympy.lambdify((r['x'], r['z']), r['u'], modules='numpy')(X, Z)
    if idx == 0:
        ax = fig.add_subplot(gs[idx, 0])
    else:
        ax = fig.add_subplot(gs[idx, 0], sharex=vel_profile_axes[0], sharey=vel_profile_axes[0])
    vel_profile_axes.append(ax)

    surface = sympy.lambdify(r['x'], r['surface'], modules='numpy')
    below_surface = (Z < surface(X))
    below_surface_mask = np.ones_like(X, dtype=float)
    below_surface_mask[~below_surface] = np.nan
    pcm = ax.pcolormesh(X/1e3, Z, scipy.constants.year * U * below_surface_mask, cmap='viridis', alpha=0.8, norm=norm)

    # Add layers
    for layer in r['layers_t0']:
        ax.plot(r['xs']/1e3, layer(r['xs']), linewidth=0.5, linestyle='--', color='k')

    ax.set_title(f'n = {r["n"]}')
    ax.set_title(vel_plot_labels[idx], loc='left', fontweight='bold', color=result_colors[idx])
    ax.set_ylabel('z [m]')

    ax.set_xlim(0, r['domain_x']/1e3)
    ax.set_ylim(0, surface(0))
    
    last_result_ax = ax # Track the bottom axis for adding the layer legend

fig.colorbar(pcm, ax=vel_profile_axes, label='Velocity magnitude [m/yr]')
vel_profile_axes[-1].set_xlabel('x [km]')

# Horizontal velocity at 90 km
ax = fig.add_subplot(gs[:, 1])
plot_pos_x = 90e3

for result_idx, res in enumerate(results):
    u_at_plot_pos = sympy.lambdify(res['z'], res['u'].subs(res['x'], plot_pos_x), modules='numpy')(res['zs'])

    for layer_idx in res['layer_solutions'].keys():
        if (layer_idx == 1):
            lbl = f'ODE Solutions for n = {res["n"]}'
        else:
            lbl = None
        ax.scatter([res['layer_solutions'][layer_idx].sol(plot_pos_x)], [res['layers_t0'][layer_idx](plot_pos_x)], label=lbl, c=result_colors[result_idx], marker=result_markers[result_idx])

    ax.plot(u_at_plot_pos*scipy.constants.year, res['zs'], linestyle='--', linewidth=1, c=result_colors[result_idx], label=f'True n = {res["n"]}')

ax.set_xlabel('Horizontal velocity [m/yr]')
ax.set_ylabel('z [m]')
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], fontsize='small')
ax.grid()
#ax.set_title(f'Horizontal Velocity Profile\nat x = {plot_pos_x/1e3} km')
ax.set_title('(d)', loc='left', fontweight='bold')

# Strain rate profiles
ax = fig.add_subplot(gs[:, 2])
for result_idx, res in enumerate(results):
    ax.scatter(np.log10(res['eff_stress']), np.log10(res['du_dz_central_diff']), label=f'n = {res["n"]}', s=2, c=result_colors[result_idx], marker=result_markers[result_idx])

ax.set_aspect(0.5)
ax.set_xlabel('log(effective stress)')
ax.set_ylabel('log(strain rate)')
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], fontsize='small')
# Set the grid spacing to 1 on both axes
ax.set_xticks(np.arange(2, 5, 1))
ax.set_yticks(np.arange(-10, -2, 1))
#ax.set_ylim(-10, -2)
ax.set_xlim(2, 4.5)
ax.grid()
#ax.set_title(f'Stress vs Strain\nalong characteristic curves')
ax.set_title('(e)', loc='left', fontweight='bold')

import matplotlib.lines as mlines
layer_line = mlines.Line2D([], [], linewidth=0.5, linestyle='--', color='k', label='Layers')
last_result_ax.legend(handles=[layer_line], bbox_to_anchor=(0.3, -0.2))

#fig.savefig('figures/example_rheology.png', dpi=500)